In [0]:
%run ./MasterNoteBooks/MNB_Sales_Emp_Dep

In [0]:
## Create a UDF to normalize NetSalary between 0 and 1 across all employees.

emp = load_employee() 
dep =load_department ()
sal = load_salary()

## Taking min and max netsalary for employees. 
min_salary = sal.agg(min("NetSalary")).collect()[0][0]
max_salary = sal.agg(max("NetSalary")).collect()[0][0]


def NormalizeSalary(NetSalary):
    try:
        if max_salary != min_salary:
            return (NetSalary - min_salary) / (max_salary- min_salary)
        else :
            return 0.0
    
    except Exception as E:
        return E

join =  (emp
         .join(dep,"EmployeeID")
         .join(sal,"EmployeeID")) 

udf_NormalizeSalary = udf(NormalizeSalary , DoubleType())

final = (
    join.withColumn("NormalizedSalary",udf_NormalizeSalary("NetSalary"))
    .withColumn("NormalizedSalary",round(col("NormalizedSalary"),2))
    .select("EmployeeID","Name","Department","NormalizedSalary")
    .where("NormalizedSalary is not null")
)

display(final)